In [1]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import re

In [2]:
pd.options.display.max_colwidth = 200

### Load and inspect data

In [3]:
loan_data_df = pd.read_csv('loan.csv', low_memory=False)
column_desc_df = pd.read_excel('Data_Dictionary.xlsx')
loan_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Columns: 111 entries, id to total_il_high_credit_limit
dtypes: float64(74), int64(13), object(24)
memory usage: 33.6+ MB


### Data Clean up

#### Delete all empty columns

In [4]:
empty_columns = loan_data_df.isna().all().sum()
print(f"Number of columns where all data is empty: {empty_columns}")
# Remove all columns with null values
loan_data_df = loan_data_df.dropna(axis=1, how='all')
# Remove all columns with 90% null values
loan_data_df = loan_data_df.loc[:, (loan_data_df.isnull().sum(axis=0) <= loan_data_df.shape[0] * 0.9)]

Number of columns where all data is empty: 54


#### Remove columns which have constant as a value. 
These will become metadata for the dataset because the dataset has only one value for them.
eg. all values in column: pymnt_plan are `n`.

In [5]:
constant_cols = []
for col in loan_data_df.columns:
    if loan_data_df[col].value_counts().count() == 1:
        constant_cols.append(col)
        print(f"Column: {col} has value: {loan_data_df[col].value_counts().index[0]}")
print(f"Columns to be removed: {constant_cols}")
loan_data_df.drop(columns=constant_cols, inplace=True) 

Column: pymnt_plan has value: n
Column: initial_list_status has value: f
Column: collections_12_mths_ex_med has value: 0.0
Column: policy_code has value: 1
Column: application_type has value: INDIVIDUAL
Column: acc_now_delinq has value: 0
Column: chargeoff_within_12_mths has value: 0.0
Column: delinq_amnt has value: 0
Column: tax_liens has value: 0.0
Columns to be removed: ['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


#### Inspecting data in columns, 
Some columns can be removed since they don't give much use to the analysis

In [6]:
print("desc, purpose, title are similar - title and desc seem to be human added and have more distinct values. Purpose has a finite list and it alone can be looked at and drop desc and title.")
print("\n-------title-----------\n", loan_data_df.title.value_counts().index, 
      # "\n-------desc----------\n", loan_data_df.desc.value_counts().index, # Too much data. skipping print
      "\n-------purpose-----------\n", loan_data_df.purpose.value_counts().index)
cols_to_drop = ['title', 'desc']
print("url column has external url for lending club, and not much use to analysys")
print(loan_data_df.url.head())
cols_to_drop.append('url')
loan_data_df.drop(columns=cols_to_drop, inplace=True) 

desc, purpose, title are similar - title and desc seem to be human added and have more distinct values. Purpose has a finite list and it alone can be looked at and drop desc and title.

-------title-----------
 Index(['Debt Consolidation', 'Debt Consolidation Loan', 'Personal Loan',
       'Consolidation', 'debt consolidation', 'Home Improvement',
       'Credit Card Consolidation', 'Debt consolidation',
       'Small Business Loan', 'Credit Card Loan',
       ...
       'Stav', 'Florida Vacation 2011', 'Ninja zx6', 'Pay off Masters Degree',
       'CREDIT CARD CONSOLIDATION LOAN 01', 'your rate is better than my rate',
       'Concession Trailer', 'gregs', 'EZover', 'JAL Loan'],
      dtype='object', length=19615) 
-------purpose-----------
 Index(['debt_consolidation', 'credit_card', 'other', 'home_improvement',
       'major_purchase', 'small_business', 'car', 'wedding', 'medical',
       'moving', 'vacation', 'house', 'educational', 'renewable_energy'],
      dtype='object')
url co

#### Dataset after removed columns

In [8]:
loan_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Data columns (total 43 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       39717 non-null  int64  
 1   member_id                39717 non-null  int64  
 2   loan_amnt                39717 non-null  int64  
 3   funded_amnt              39717 non-null  int64  
 4   funded_amnt_inv          39717 non-null  float64
 5   term                     39717 non-null  object 
 6   int_rate                 39717 non-null  object 
 7   installment              39717 non-null  float64
 8   grade                    39717 non-null  object 
 9   sub_grade                39717 non-null  object 
 10  emp_title                37258 non-null  object 
 11  emp_length               38642 non-null  object 
 12  home_ownership           39717 non-null  object 
 13  annual_inc               39717 non-null  float64
 14  verification_status   

In [9]:
column_desc_df = column_desc_df[column_desc_df.LoanStatNew.isin(loan_data_df.columns)]
column_desc_df.reset_index(inplace=True)

In [10]:
column_desc_df.loc[:, ['LoanStatNew', 'Description']]

,LoanStatNew,Description
0,addr_state,The state provided by the borrower in the loan application
1,annual_inc,The self-reported annual income provided by the borrower during registration.
2,collection_recovery_fee,post charge off collection fee
3,delinq_2yrs,The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years
4,dti,"A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly inc..."
5,earliest_cr_line,The month the borrower's earliest reported credit line was opened
6,emp_length,Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.
7,emp_title,The job title supplied by the Borrower when applying for the loan.*
8,funded_amnt,The total amount committed to that loan at that point in time.
9,funded_amnt_inv,The total amount committed by investors for that loan at that point in time.


#### Convert the columns to correct datatypes

In [11]:
def convert_emp_length(l):
    if pd.isna(l):
        return np.nan
    elif l == '< 1 year':
        return 0
    elif l == '10+ years':
        return 10
    else:
        return int(str(l)[0])

In [12]:
# numbers
loan_data_df.term               = loan_data_df.term.apply(lambda x: int(str(x)[:3]))
loan_data_df.emp_length         = loan_data_df.emp_length.apply(convert_emp_length) 
# percentages
loan_data_df.int_rate           = loan_data_df.int_rate.apply(lambda x: float(str(x)[:-1])/100)
loan_data_df.revol_util         = loan_data_df.revol_util.apply(lambda x: (float(str(x)[:-1])/100) if str(x) != 'nan' else x)
# dates
loan_data_df.issue_d            = pd.to_datetime(loan_data_df.issue_d, format='%b-%y')
loan_data_df.earliest_cr_line   = pd.to_datetime(loan_data_df.earliest_cr_line, format='%b-%y')
loan_data_df.last_pymnt_d       = pd.to_datetime(loan_data_df.last_pymnt_d, format='%b-%y')
loan_data_df.last_credit_pull_d = pd.to_datetime(loan_data_df.last_credit_pull_d, format='%b-%y')


In [13]:
loan_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Data columns (total 43 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   id                       39717 non-null  int64         
 1   member_id                39717 non-null  int64         
 2   loan_amnt                39717 non-null  int64         
 3   funded_amnt              39717 non-null  int64         
 4   funded_amnt_inv          39717 non-null  float64       
 5   term                     39717 non-null  int64         
 6   int_rate                 39717 non-null  float64       
 7   installment              39717 non-null  float64       
 8   grade                    39717 non-null  object        
 9   sub_grade                39717 non-null  object        
 10  emp_title                37258 non-null  object        
 11  emp_length               38642 non-null  float64       
 12  home_ownership           39717 n

### Remove/add value to rows with null values in certain columns

In [20]:
loan_data_df.isna().sum()[loan_data_df.isna().sum() > 0]

emp_title                  2459
emp_length                 1075
mths_since_last_delinq    25682
revol_util                   50
last_pymnt_d                 71
last_credit_pull_d            2
pub_rec_bankruptcies        697
dtype: int64

In [31]:
# Remove rows with very few (<100) NaNs:
cols_with_few_nan_rows = list(loan_data_df.isna().sum()[(loan_data_df.isna().sum() < 100) & (loan_data_df.isna().sum() > 0)].index)
for col in cols_with_few_nan_rows:
    loan_data_df.drop(loan_data_df[loan_data_df[col].isna()].index, inplace=True)

emp_title                  2446
emp_length                 1067
mths_since_last_delinq    25600
pub_rec_bankruptcies        696
dtype: int64

In [33]:
print("Columns with non-zero NaN values left:")
loan_data_df.isna().sum()[loan_data_df.isna().sum() > 0]

Columns with non-zero NaN values left:


emp_title                  2446
emp_length                 1067
mths_since_last_delinq    25600
pub_rec_bankruptcies        696
dtype: int64

### Find and categorise columns of interest